# Merge thermo libraries
This script is used to merge different thermo libraries generated from ARC

### Necessary packages

In [1]:
import logging
import os
import shutil

from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase

from toolbox.thermolib import find_thermo_libs, read_thermo_lib_by_path, \
                              merge_thermo_lib

%matplotlib inline
%load_ext autoreload
%autoreload 2

logger = logging.getLogger()
logger.setLevel(logging.INFO)

### [OPTIONAL] Assign a log file to record all the changes 

In [2]:
fh = logging.FileHandler('thermo_lib_tools.log', mode="a+")
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
logger.addHandler(fh)

### Load a RMG database instance
You may need to add some RMG built-in libraries for your purpose. The generated `thermo_database` will be used in the later section.

In [3]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'],
    thermo_libraries = [],  # Can add other library if necessary
    kinetics_families = "default",
    reaction_libraries = [],
    kinetics_depositories = ['training'],
)

thermo_database = database.thermo

INFO:root:Loading thermodynamics group database from /Users/xiaorui/Apps/RMG-database/input/thermo/groups...
INFO:root:Loading transport library from GRI-Mech.py in /Users/xiaorui/Apps/RMG-database/input/transport/libraries...
INFO:root:Loading transport library from OneDMinN2.py in /Users/xiaorui/Apps/RMG-database/input/transport/libraries...
INFO:root:Loading transport library from NOx2018.py in /Users/xiaorui/Apps/RMG-database/input/transport/libraries...
INFO:root:Loading transport library from PrimaryTransportLibrary.py in /Users/xiaorui/Apps/RMG-database/input/transport/libraries...
INFO:root:Loading transport group database from /Users/xiaorui/Apps/RMG-database/input/transport/groups...
INFO:root:Loading frequencies group database from /Users/xiaorui/Apps/RMG-database/input/statmech/groups...
INFO:root:Loading Platts additivity group database from /Users/xiaorui/Apps/RMG-database/input/solvation/groups...


### Input
Merge the libraries from working directory and save the new library into the save path. If the save path is orignially a library, then merge everything on this library.
- `save_path` (str): the full path to save the new merged library
- `work_dir` (str): indicate where your thermo libraries (generated by ARC). Do not need to provide the full path to the file because the script will search for the thermolibrary.

In [4]:
save_path = '../media/thermo library/compare_thermo_library_entries/new_lib.py'
work_dir = '../media/thermo library/compare_thermo_library_entries/'

In [5]:
# Get the thermo libraries under work_dir
thermo_lib_list = find_thermo_libs(work_dir)
for thermo_lib in thermo_lib_list:
    read_thermo_lib_by_path(thermo_lib, thermo_database)

# Generate the new library
if not os.path.isfile(save_path):
    shutil.copy(thermo_lib, save_path)
read_thermo_lib_by_path(save_path, thermo_database)
base_lib = thermo_database.libraries[save_path]    

# Combine the thermo libraries 
for thermo_lib in thermo_lib_list:
    library_to_add = thermo_database.libraries[thermo_lib]
    merge_thermo_lib(base_lib, library_to_add)
    
# Save the libs
base_lib.save(base_lib.label)
for thermo_lib in thermo_lib_list:
    lib_to_add = thermo_database.libraries[thermo_lib]
    lib_to_add.save(thermo_lib)

INFO:root:Find thermo library at ../media/thermo library/compare_thermo_library_entries/fake_arc_project2/output/RMG libraries/thermo/BA.py
INFO:root:Find thermo library at ../media/thermo library/compare_thermo_library_entries/fake_arc_project1/output/RMG libraries/thermo/BA.py
INFO:root:Loading thermodynamics library BA.py from ../media/thermo library/compare_thermo_library_entries/fake_arc_project2/output/RMG libraries/thermo ...
INFO:root:Loading thermodynamics library BA.py from ../media/thermo library/compare_thermo_library_entries/fake_arc_project1/output/RMG libraries/thermo ...
INFO:root:Loading thermodynamics library new_lib.py from ../media/thermo library/compare_thermo_library_entries ...


[OPTIONAL] Remove tags

In [ ]:
for thermo_lib in thermo_lib_list:
    lib_to_mod = thermo_database.libraries[thermo_lib]
    for spc in lib_to_mod.entries.values():
        spc.shortDesc = ''
lib_to_mod.save(thermo_lib)